To finetune the Resnet model, we first generate adversarial training images and adversarial validation images by the following commands:
```
python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_train_images'
python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_val_images' --seed 0
```
The params are all default.
Then we use `adv_train_images` to train the Resnet model and save it.


In [2]:
!python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_train_images'
!python launch_resnet_attack.py --batch_num 20 --batch_size 100 --results 'adv_val_images' --seed 0

Loading model...
Loading data...
===Launching PGD attack on 20 batches of data===
Attack configs: eps = 0.03137254901960784, alpha = 0.00784313725490196, steps = 20, batch size = 100
100%|███████████████████████████████████████████| 20/20 [00:50<00:00,  2.54s/it]
Accuracy on original images: 92.7%
Accuracy on adversarial images: 2.0500000000000003%
Loading model...
Loading data...
===Launching PGD attack on 20 batches of data===
Attack configs: eps = 0.03137254901960784, alpha = 0.00784313725490196, steps = 20, batch size = 100
 60%|█████████████████████████▊                 | 12/20 [00:34<00:15,  2.00s/it]/home/guest/zsf/conda3/envs/xz/lib/python3.11/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
100%|███████████████████████████████████████████| 20/20 [00:49<00:00,  2.46s/it]
Accuracy on original images: 91.75%
Accuracy on adversarial images: 2.75%


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, TensorDataset

ADV_TRAIN_PATH = './results/adv_train_images'
BATCH_SIZE = 100
BATCH_NUM = 20
torch.manual_seed(1234)

checkpoint = torch.load(ADV_TRAIN_PATH)
adv_images = checkpoint['adv_images']
labels = checkpoint['labels']

dataset = TensorDataset(adv_images, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')

torch.save(model.state_dict(), 'results/resnet50_finetuned.pth')

/tmp/ipykernel_1476066/1210653890.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(ADV_TRAIN_PATH)


Epoch [1/10], Loss: 4.3065
Epoch [2/10], Loss: 1.4645
Epoch [3/10], Loss: 0.8922
Epoch [4/10], Loss: 0.6378
Epoch [5/10], Loss: 0.4642
Epoch [6/10], Loss: 0.3552
Epoch [7/10], Loss: 0.2739
Epoch [8/10], Loss: 0.2285
Epoch [9/10], Loss: 0.1837
Epoch [10/10], Loss: 0.1546


We load the finetuned model and use validation images to test it.

The accuracy on adversarial validation images reaches 73.05%, higher than the model without finetuned (2.75%).

In [4]:
model = resnet50(pretrained=False)
model.load_state_dict(torch.load('results/resnet50_finetuned.pth'))
model = model.to(device)

ADV_VAL_PATH = './results/adv_val_images'
BATCH_SIZE = 100
BATCH_NUM = 20

checkpoint = torch.load(ADV_VAL_PATH)
val_images = checkpoint['adv_images']
labels = checkpoint['labels']

dataset = TensorDataset(val_images, labels)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

model.eval()
correct = 0
total = 0

for inputs, labels in dataloader:
  inputs, labels = inputs.to(device), labels.to(device)
  with torch.no_grad():
    outputs = model(inputs).softmax(1)
    predictions = outputs.argmax(dim=1)
  correct += torch.sum(predictions == labels).item()
  total += len(labels)


acc = correct / total
print(f"Accuracy on adversarial validation images: {acc * 100}%")

/home/guest/zsf/conda3/envs/xz/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/guest/zsf/conda3/envs/xz/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_1476066/1684913400.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be

Accuracy on adversarial validation images: 73.05%
